In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
# change working directory otherwise accessing data folder fails
os.chdir('..')
from evaluate import EvaluateArgs, evaluate
from tome_sam.utils.tome_presets import SAMToMeSetting, BSMToMe, ToMeConfig, PiToMe
from flops import get_flops

# Utilities for visualizing the results

/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/siwei/miniforge3/envs/tome_sam/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, 

Pure SAM

In [2]:
evaluate_args = EvaluateArgs(
    dataset="dis",
    output="",
    model_type="vit_b",
    checkpoint="checkpoints/sam_vit_b_01ec64.pth",
    device="mps",
    seed=0,
    input_size=[1024, 1024],
    batch_size=1,
    multiple_masks=False,
    tome_setting = None,
)

iou_results = evaluate(evaluate_args)
print(iou_results)

--- Create valid dataloader with dataset dis ---
------------------------------ valid --------------------------------
--->>> dataset:  DIS5K-VD <<<---
-im- DIS5K-VD ./data/DIS5K/DIS-VD/im :  470
-gt- DIS5K-VD ./data/DIS5K/DIS-VD/gt :  470
--- Valid dataloader with dataset dis created ---
--- Create SAM vit_b with token merging in layers None ---


/Users/siwei/Desktop/tome_sam/tome_sam/build_tome_sam.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


--- Start evaluation ---
valid dataloader length: 470


/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1729647065806/work/aten/src/ATen/native/TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  [  0/470]  eta: 0:10:01  mask_iou: 0.7190 (0.7190)  boundary_iou: 0.4607 (0.4607)  time: 1.2808  data: 0.0673
  [200/470]  eta: 0:04:05  mask_iou: 0.5820 (0.5353)  boundary_iou: 0.5071 (0.4591)  time: 0.8618  data: 0.0721
  [400/470]  eta: 0:01:05  mask_iou: 0.6821 (0.5435)  boundary_iou: 0.4338 (0.4620)  time: 0.9277  data: 0.0882
  [469/470]  eta: 0:00:00  mask_iou: 0.5600 (0.5415)  boundary_iou: 0.4693 (0.4611)  time: 0.7926  data: 0.0568
 Total time: 0:07:16 (0.9296 s / it)
Averaged stats: mask_iou: 0.5600 (0.5415)  boundary_iou: 0.4693 (0.4611)
{'mask_iou': 0.5414679513996238, 'boundary_iou': 0.46114134099164045}


In [4]:
evaluate_args = EvaluateArgs(
    dataset="dis",
    output="",
    model_type="vit_b",
    checkpoint="checkpoints/sam_vit_b_01ec64.pth",
    device="cpu",
    seed=0,
    input_size=[1024, 1024],
    batch_size=1,
    multiple_masks=False,
    tome_setting = None,
)

flops_per_image = get_flops(evaluate_args)
print(f'flops per image: {flops_per_image}')

--- Create valid dataloader with dataset dis ---
------------------------------ valid --------------------------------
--->>> dataset:  DIS5K-VD <<<---
-im- DIS5K-VD ./data/DIS5K/DIS-VD/im :  470
-gt- DIS5K-VD ./data/DIS5K/DIS-VD/gt :  470
--- Valid dataloader with dataset dis created ---
--- Create SAM vit_b with token merging in layers None ---
--- Start flop count analysis ---


flops per image: 488.2478736639998


ToMe

In [2]:
test_bsm_setting: SAMToMeSetting = {
    2: ViTToMeConfig(
        kv=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        ),
        q=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        )
    ),
    5: ViTToMeConfig(
        kv=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        ),
        q=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        )
    ),
    8: ViTToMeConfig(
        kv=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        ),
        q=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        )
    ),
    11: ViTToMeConfig(
        kv=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        ),
        q=ToMeConfig(
            mode='bsm',
            params=BSMToMe(r=0.6, sx=2, sy=2)
        )
    ),
}
evaluate_args = EvaluateArgs(
    dataset="dis",
    output="",
    model_type="vit_b",
    checkpoint="checkpoints/sam_vit_b_01ec64.pth",
    device="mps",
    seed=0,
    input_size=[1024, 1024],
    batch_size=1,
    multiple_masks=False,
    tome_setting = test_bsm_setting,
)

iou_results = evaluate(evaluate_args)
print(iou_results)


--- Create valid dataloader with dataset dis ---
------------------------------ valid --------------------------------
--->>> dataset:  DIS5K-VD <<<---
-im- DIS5K-VD ./data/DIS5K/DIS-VD/im :  470
-gt- DIS5K-VD ./data/DIS5K/DIS-VD/gt :  470
--- Valid dataloader with dataset dis created ---
--- Create SAM vit_b with token merging in layers {2: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2))), 5: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2))), 8: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2))), 11: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)))} ---


/Users/siwei/Desktop/tome_sam/tome_sam/build_tome_sam.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


--- Start evaluation ---
valid dataloader length: 470


/Users/siwei/miniforge3/envs/tome_sam/lib/python3.10/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1729647065806/work/aten/src/ATen/native/TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/siwei/Desktop/tome_sam/tome_sam/tome_algo/tome/merge.py:126: UserWarning: The operator 'aten::scatter_reduce.two_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1729647065806/work/aten/src/ATen/mps/MPSFallback.mm:13.)
  dst = dst.scatter_reduce(-2, dst_idx.expand(n, r, c), src, reduce=mode)


  [  0/470]  eta: 0:10:11  mask_iou: 0.8027 (0.8027)  boundary_iou: 0.6548 (0.6548)  time: 1.3019  data: 0.0633
  [200/470]  eta: 0:04:07  mask_iou: 0.5901 (0.4792)  boundary_iou: 0.4899 (0.4082)  time: 0.8640  data: 0.0675
  [400/470]  eta: 0:01:05  mask_iou: 0.4806 (0.4933)  boundary_iou: 0.4445 (0.4147)  time: 0.9404  data: 0.0900
  [469/470]  eta: 0:00:00  mask_iou: 0.5428 (0.4907)  boundary_iou: 0.4032 (0.4133)  time: 0.8061  data: 0.0553
 Total time: 0:07:21 (0.9383 s / it)
Averaged stats: mask_iou: 0.5428 (0.4907)  boundary_iou: 0.4032 (0.4133)
{'mask_iou': 0.4906588361156814, 'boundary_iou': 0.4133173046541157}


In [3]:
evaluate_args = EvaluateArgs(
    dataset="dis",
    output="",
    model_type="vit_b",
    checkpoint="checkpoints/sam_vit_b_01ec64.pth",
    device="cpu",
    seed=0,
    input_size=[1024, 1024],
    batch_size=1,
    multiple_masks=False,
    tome_setting = test_bsm_setting,
)

flops_per_image = get_flops(evaluate_args)
print(f'flops per image: {flops_per_image}')

--- Create valid dataloader with dataset dis ---
------------------------------ valid --------------------------------
--->>> dataset:  DIS5K-VD <<<---
-im- DIS5K-VD ./data/DIS5K/DIS-VD/im :  470
-gt- DIS5K-VD ./data/DIS5K/DIS-VD/gt :  470
--- Valid dataloader with dataset dis created ---
--- Create SAM vit_b with token merging in layers {2: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2))), 5: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2))), 8: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2))), 11: ViTToMeConfig(kv=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)), q=ToMeConfig(mode='bsm', params=BSMToMe(r=0.6, sx=2, sy=2)))} ---
--- Start flop count analysis ---


flops per image: 413.6002552319999


PiToMe

In [6]:
test_pitome_setting = SAMToMeSetting = {
    2: ViTToMeConfig(
        kv=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        ),
        q=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        )
    ),
    5: ViTToMeConfig(
        kv=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        ),
        q=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        )
    ),
    8: ViTToMeConfig(
        kv=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        ),
        q=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        )
    ),
    11: ViTToMeConfig(
        kv=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        ),
        q=ToMeConfig(
            mode='pitome',
            params=PiToMe(r=0.5, margin=0.5, alpha=1.0)
        )
    ),
}

evaluate_args = EvaluateArgs(
    dataset="dis",
    output="",
    model_type="vit_b",
    checkpoint="checkpoints/sam_vit_b_01ec64.pth",
    device="mps",
    seed=0,
    input_size=[1024, 1024],
    batch_size=1,
    multiple_masks=False,
    tome_setting = test_pitome_setting,
)

iou_results = evaluate(evaluate_args)
print(iou_results)

--- Create valid dataloader with dataset dis ---
------------------------------ valid --------------------------------
--->>> dataset:  DIS5K-VD <<<---
-im- DIS5K-VD ./data/DIS5K/DIS-VD/im :  470
-gt- DIS5K-VD ./data/DIS5K/DIS-VD/gt :  470
--- Valid dataloader with dataset dis created ---
--- Create SAM vit_b with token merging in layers {2: ViTToMeConfig(kv=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0)), q=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0))), 5: ViTToMeConfig(kv=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0)), q=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0))), 8: ViTToMeConfig(kv=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0)), q=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0))), 11: ViTToMeConfig(kv=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alpha=1.0)), q=ToMeConfig(mode='pitome', params=PiToMe(r=0.5, margin=0.5, alph

In [2]:
evaluate_args = EvaluateArgs(
    dataset="dis",
    output="",
    model_type="vit_b",
    checkpoint="checkpoints/sam_vit_b_01ec64.pth",
    device="cpu",
    seed=0,
    input_size=[1024, 1024],
    batch_size=1,
    multiple_masks=False,
    tome_setting = test_pitome_setting,
)
flops_per_image = get_flops(evaluate_args)
print(f'flops per image: {flops_per_image}')

NameError: name 'test_pitome_setting' is not defined